In [ ]:
# SETUP

# These lines import the Numpy and Datascience modules.
import numpy as np
from datascience import *
from prob140 import *

# These lines do some fancy plotting magic
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

from scipy import stats
from client.api.assignment import load_assignment
autograder = load_assignment('main.ok')

# Lab 3: Riffle Shuffles and Randomness
In probability theory we often assume that a "well shuffled standard deck" is one in which all 52! permutations of the cards are equally likely. But how to actually achieve this "well shuffled" state with a physical deck?

Partly, the answer depends on the method of shuffling. In this lab, you will study one kind of shuffle, called the *riffle shuffle* (see roughly 0:25-0:30 of this [YouTube video](https://www.youtube.com/watch?v=KcH7zpd5viM)). Your goal will be to see how many times you have to riffle shuffle a standard deck so that the cards get "all mixed up".

Disclaimer: 52! factorial is a large number, and you won't be getting close to simulating the uniform distribution on that many possible outcomes. You'll do something much more tractable that still allows you to get a sense of the randomness in repeated riffle shuffling.

Run the code in the following cell. You will need it for this lab.

In [ ]:
def shuffle_deck(deck):
    n = len(deck)
    new_deck = []
    
    newSplit = np.random.binomial(n,.5) # Picks a split in the deck
    if newSplit == n or newSplit == 0:
        return np.array(deck)
    
    startA = newSplit
    startB = n-1
    
    add_to_deck = new_deck.append # This is for speed and to stop having to look up
    while (startA+1) + (startB-newSplit) > 0:
        pFromA = (startA+1)/((startA)+1+(startB-newSplit))
        
        if np.random.random() < pFromA:       # Accept from Pile 1
            add_to_deck(deck[startA])
            startA -= 1
        else:                              #  Accept from Pile 2
            add_to_deck(deck[startB])
            startB -= 1
            
    return np.array(list(reversed(new_deck)))

def shuffler(n,r):
    deck = list(range(1, n+1)) # Starts with {0 ... n-1}
    for iteration in range(r):
        deck = shuffle_deck(deck)
    return np.array(deck)

def starting_deck(n):
    return np.arange(1, n+1)

## Part 1: The Shuffle ##
Here is a probabilistic model that has been shown to roughly reflect riffle shuffles performed by humans. We will assume that the cards have been labeled 1 through 52.

### One Riffle Shuffle ###
- Start with a deck in which the cards are perfectly ordered in sequence from 1 through 52. We'll say that Card 1 is at the top and Card 52 is at the bottom.
- Cut the deck roughly in half by picking a binomial (52, 0.5) spot. All cards up to and including that spot constitute the "left side" of the deck. All cards after that spot form the "right side".
- Now *drop* cards one by one to form a permuted deck; the first card you drop will be the *bottom* card of the permuted deck, that is, Position 52 of the permuted deck. Here is how you will drop the cards:
    - Pick "left side" or "right side" with probability proportional to size. That is, if there are $n$ cards on the left side and $m$ on the right, pick the left size with chance $n/(n+m)$ and the right side with the remaining chance $m/(n+m)$. Yes, $n+m = 52$ at this stage. This "size biased sampling" takes into account what happens physically when the two sides aren't equal.
    - Drop the bottom card of the side you picked. This becomes the bottom card of the permuted deck, and the two sides now have a total of $n+m-1$ cards.
    - Repeat: pick left or right again with probability proportional to size, and drop the bottom card of the picked side, on top of the card that has already been dropped.
    - Repeat till you have dropped all 52 cards.

That's one complete riffle shuffle. Now let the permuted deck be your starting point, and do another riffle shuffle. Once that's done, do it again ... You get the picture. This lab is about when you should stop.

#### A Small Deck, and Just One Riffle Shuffle ####
To confirm that you understand the shuffle, imagine that you start with a deck of 7 cards. The cards are labeled 1, 2, 3, 4, 5, 6, and 7 and are in that order.

- Cut the deck at a binomial (7, 0.5) point as described above. Let's assume the binomial random number came out to be 2. Then the left side is $\{1, 2\}$ with Card 1 on top and Card 2 below. The right side is $\{ 3, 4, 5, 6, 7\}$ with Card 3 on top and the rest in sequence below.
- Pick a side with P(left) = 2/7. Suppose you ended up picking the right side. Then Card 7 drops.
    - Suppose the six subsequent choices are "left, right, right, right, left, right'. Then the permuted deck is $\{3, 1, 4, 5, 6, 2, 7 \}$.
    
Notice how you can spot increasing subsequences from each side. By analyzing these, you can reconstruct the moves.

### 1.1: Unshuffle this Deck ###
Suppose I started with the deck $\{1, 2, 3, 4, 5, 6, 7 \}$ in that order, performed one riffle shuffle, and ended up with $\{1, 4, 5, 6, 2, 7, 3\}$.

What was the value of the binomial (7, 0.5) variable used to cut the deck, and what was the sequence of 7 "left" and "right" choices?

*Provide your answer and reasoning in this Markdown cell.*

### 1.2: Using `shuffler` ###

Coding the riffle shuffle is not hard but it requires some care. You have to account for which side gets exhausted first, what happens when the binomial cut is at the ends, and so on. There isn't enough time for that during this lab. So we have written the code for you.

- `shuffler(deck_size, num_shuffles)` returns an array that is the shuffled deck of `deck_size` cards, after `num_shuffles` riffle shuffles. The starting  state of the deck is the integers 1 through `deck_size` in sequence, with 1 on top.

It is not required to understand this code to complete the rest of the lab. Make sure you ran the code cell at the beginning of the lab

Use `shuffler` to perform 1 riffle shuffle of a deck of 10 cards. Give the value of the binomial cut and the left-right sequence.

**Note.** Once you have written your answer, please don't rerun the code cell because it will most likely result in a different shuffle.

*Provide your answer and reasoning in this Markdown cell.*

### Question 1.2

Use `shuffler` to perform 3 riffle shuffles of a deck of of 10 cards.

Use `shuffler` to riffle shuffle a standard deck of 52 cards 6 times. The last line of your code should evaluate to the top card of the shuffled deck.

In [ ]:
permuted = ...
permuted.item(0)

## Part 2: The Top Card After 1 Shuffle ##
For $i = 1, 2, \ldots , 52$, let $X_i$ be the label of the card in Position $i$ of the deck. Before the shuffling process starts, $P(X_i = i) = 1$ for each $i$. 

If the deck is "well shuffled", the joint distribution of $(X_1, X_2, \ldots, X_{52})$ should be uniform on all 52! permutations. But if the deck is not well shuffled, then the joint distribution will be some other distribution on 52! possible outcomes. That could be a very complicated object. Let's not get into that level of detail in this lab.

Instead, let's just focus on the marginal distribution of $X_1$ after one riffle shuffle. $X_1$ is the card that appears at the top of the permuted deck.


### Question 2.1

What are the possible values of $X_1$?

*Provide your answer and reasoning in this Markdown cell.*

### Question 2.2 

Simulate the distribution of $X_1$ after 1 riffle shuffle. Use 5000 repetitions.

In [ ]:
reps = 5000
deck_size = 52
num_shuffles = 1

# array to store the top card from each simulation
top_card_label = make_array()

for i in range(reps):
    
    # the permuted deck after one riffle shuffle
    permuted = ... 
    
    # top card of shuffled deck
    label = ... 
    
    # augment the storage array
    top_card_label = ...


### Question 2.3

We can now call [`emp_dist(array)`](https://probability.gitlab.io/prob140/html/_autosummary/prob140.emp_dist.html#prob140.emp_dist) to create an empirical distribution. Create an empirical distribution of $X_1$

In [ ]:
empirical_dist_x1 = ...
empirical_dist_x1

Run the cell below to plot the empirical distribution of $X_{1}$.

In [ ]:
Plot(empirical_dist_x1)
plt.xlim(0, 52);

### Question 2.4
Your graph should show a strange-looking distribution that appears to be in two parts. It can be thought of as a *mixture* of two distributions: 

- With probability about 1/2, it's a distribution that puts all its probability on just one value.
- With the remaining probability of about 1/2, it's a different distribution that you should identify.

Think about how a label ends up at the top of the deck after a single riffle shuffle, and explain why the distribution of $X_1$ looks the way it does. Identify the two pieces of the distribution and check that the areas of the bars are consistent with your explanation. You do not have to prove anything mathematically but your reasoning should be convincing.

*Provide your answer and reasoning in this Markdown cell.*

## Part 3: The Card in Any Position After Any Number of Shuffles

Now you will write a function that takes three arguments:

- `position`: a position between 1 and 52 (inclusive)
- `num_shuffles`: the number of times to riffle shuffle the deck
- `reps`: the number of repetitions of the simulation

and returns the empirical distribution of the label of the card at the given `position` after `num_shuffles` riffle shuffles of a deck of 52 cards starting with a perfectly ordered deck, based on `reps` repetitions of the sampling process.

### Question 3.1

Fill in the function `empirical_dist_x` so that it performs as described above. Just follow steps analogous to those in the simulation in Part 2. 

In [ ]:

deck_size = 52

def empirical_dist_x(position_number,num_shuffles,reps):
    
    location_of_x = make_array()
    
    for i in range(reps):
        
        permuted = ...
    
        card_label = ...

        location_of_x = ...

    return emp_dist(location_of_x)

### Question 3.2

By calling `empirical_dist_x` with the appropriate arguments, you can now easily display the empirical marginal distribution of the label at any specific position after any number of shuffles. 

We have played around with the number of repetitions to fine tune accuracy. In what follows, please use the number of repetitions provided in the question.

Generate the empirical distribution of the 25th card after 1 shuffle using the function that you have just written. 5000 reptitions should be enough. The display will look like a table, not a histogram.

In [ ]:

empirical_dist_x25 = ...
empirical_dist_x25

### Question 3.3

Now let's plot the distributions of the 1st card and the 25th card after 1 shuffle and 5000 repetitions. Explain what you see.

The limits of the x-axis should be up to 52

In [ ]:
Plots("$X_1$", ..., "$X_{25}$", ...)
plt.xlim(-1, 52);

### Question 3.4

Explain why the distribution of the label at Position 25 is what you see. Does either $X_1$ or $X_{25}$ appear to be uniformly distributed over all the cards after 1 riffle shuffle?

*Provide your answer and reasoning in this Markdown cell.*

# Part 4: Total Variation Distance

As we discovered in Part 3, after one shuffle, certain cards are very likely to be in specific spots. In a well shuffled deck, we want any card to be equally likely to be at any position. Thus, we will try to figure out how many shuffles it will take such that in each position in the shuffled deck, all 52 cards are close to equally likely.

In other words, for each position $i$, we will try to figure out how many times we have to shuffle so that the marginal distribution of $X_i$ is close to the uniform distribution on $\{1, 2, \ldots , 52\}$.

This can be measured by the total variation distance:

$$
TVD(\text{distribution of } X_i, \text{uniform distribution on 1 through 52})
~ = ~ 
\frac{1}{2} \sum_{k=1}^{52} |P(X_i = k) - \frac{1}{52}| 
$$

### Question 4.1

Write the function `tvd_emp_dist_uniform` which takes in a Distribution object with possible values $1, 2, \ldots , 52$ and returns the total variation distance between that distribution and the uniform distribution on $\{1, 2, \ldots , 52\}$.

Remember that you can access a column of a Table as an array by calling `table.column(column_name)`.

In [ ]:

def tvd_emp_dist_uniform(dist):
    
    proportions = dist.column("Proportion")
    
    return ...

The following cell defines a function `tvd_vs_shuffles` that takes in a position in the deck and plots total variation distance as follows:
- On the horizontal axis is the number of riffle shuffles. It goes from 2 to 10.
- At each number of shuffles $s$, the height of the point is the TVD between the empirical distribution of the card at the specified position and the uniform distribution, after $s$ riffle shuffles.

**Please run the cell below.**

In [ ]:
NUM_CARDS_IN_DECK = 52
from collections import Counter

def fast_tvd(list_of_seen):
    l = list(Counter(list_of_seen).values())
    l.extend([0]*(52-len(l)))
    l = np.array(l) / len(list_of_seen)
    return 0.5*np.sum(np.abs(l-1/52))

def tvd_vs_shuffles(whichCard,minShuffles=2,maxShuffles=10,reps=5000):
    assert 0 <= whichCard < NUM_CARDS_IN_DECK, "Card not in range"
    decks = [starting_deck(NUM_CARDS_IN_DECK) for _ in range(reps)]
    tvds = np.array([])
    for shuffle_num in range(maxShuffles+1):
        values = [deck[whichCard] for deck in decks]
        tvds = np.append(tvds,fast_tvd(values))
        for deck_num in range(reps):
            decks[deck_num] = shuffle_deck(decks[deck_num])
  
    plt.plot(np.arange(minShuffles,maxShuffles+1),tvds[minShuffles:],label='Card %d'%whichCard)
    plt.xlabel("Number of Shuffles")
    plt.ylabel("TVD")
    plt.title('Position %d'%whichCard)

### Question 4.2

If you call `tvd_vs_shuffles` two times in the same cell with different positions as the arguments, the two plots will be overlaid.

So call `tvd_vs_shuffles` appropriately to draw overlaid plots of the TVDs at Positions 1 and 25.

In [ ]:
tvd_vs_shuffles(...)
tvd_vs_shuffles(...)

plt.legend();

### Question 4.3

After how many shuffles do you see both the TVDs settling down to be small? Is your observation consistent with the claim that it takes about 7 riffle shuffles to get a well shuffled deck?

*Provide your answer and reasoning in this Markdown cell.*

# Part 5: Heat Maps


Thus far, we have been observing the positions one at a time. Indeed, we have only really examined two positions: 1 and 25.

A heat map allows us to visualize which label is in which position, for all the labels and positions at one.

Calling `draw_heat_map(num_shuffles)` will draw a square heat map. 
- The x-axis represents the position $i$ in the shuffled deck.
- The y-axis represents the label ($j$) of a card after `num_shuffles` riffle shuffles. 
- The color at the point $(i, j)$ shows the proportion of times that card labeled $j$ ended up at position $i$ after `num_shuffles` riffle shuffles.

Note: Keep an eye on the color scale on the right. It changes between plots.

In [ ]:
NUM_CARDS_IN_DECK = 52
def heat_map(end=10,reps=5000):
    decks = [starting_deck(NUM_CARDS_IN_DECK) for _ in range(reps)]
    heatmaps = []
    for shuffle_num in range(end):
        grid_of_ps = np.zeros((NUM_CARDS_IN_DECK,NUM_CARDS_IN_DECK))
        for i in range(reps):
            for card,location in enumerate(decks[i]):
                grid_of_ps[location-1][card-1] +=1
            decks[i] = shuffle_deck(decks[i])

        grid_of_ps /= reps
        heatmaps.append(grid_of_ps)
    return heatmaps

def showAllHeatMaps(heatmaps):
    fig, axes = plt.subplots(nrows=(len(heatmaps)//3), ncols=3)
    if len(heatmaps)//3 > 1:
        axes = sum([list(ax) for ax in axes],[])
    for n,(pic,ax) in enumerate(zip(heatmaps,axes)):
        image = ax.imshow(pic,interpolation='nearest', origin='lower')
        image.set_clim(0,.1)
        ax.axis('off')

def draw_heat_map(num_shuffles,**kwargs):
    hm = heat_map(num_shuffles+1,**kwargs)[-1]
    plt.imshow(hm,interpolation='nearest', origin='lower')
    
    if num_shuffles > 3:
        max_prob = 0.035
    else:
        max_prob = np.amax(hm)
    plt.clim(-.01,max_prob)
    plt.colorbar()
    plt.title('Empirical P($X_i=j$) after %d shuffles'%num_shuffles)
    plt.xlabel('Position (i)')
    plt.ylabel('Card Number (j)')

### Question 5.1

Run the cell below to generate a heat map for 0 shuffles; that is, before any shuffling has been done

In [ ]:
draw_heat_map(0)

Why is there a yellow diagonal line? Why is the background purple?

*Provide your answer and reasoning in this Markdown cell.*

### Question 5.2

Run the cell below to generate the heat map for 1 shuffle

In [ ]:
draw_heat_map(1)

(a) Why are the corners (0,0) and (52,52) the only spots on the heat map that have non-purple colors?

[Hint: Go back and look at your histogram in Part 1.]

(b) Explain why there is a short lighter blue vertical strip near ($i=1$, $j=25$).

*Provide your answer and reasoning in this Markdown cell.*

### Question 5.3

How many shuffles does it take until the entire heat map is entirely one color? You may create as many new cells as you want to experiment with different values

*Provide your answer and reasoning in this Markdown cell.*

Make sure you have saved your file before exporting to gradescope pdf!

In [ ]:
I_saved_my_file = False

In [ ]:
_ = autograder.grade('q1')

In [ ]:
# For your convenience, you can run this cell to run all the tests at once!
import os
_ = [autograder.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]

In [ ]:
import gsExport
gsExport.generateSubmission()

### References ###
- [In Shuffling Cards, 7 is Winning Number](http://www.nytimes.com/1990/01/09/science/in-shuffling-cards-7-is-winning-number.html?pagewanted=all)
- [Shuffling Cards and Stopping Times](http://statweb.stanford.edu/~cgates/PERSI/papers/aldous86.pdf)
- [Trailing the Dovetail Shuffle to its Lair](http://statweb.stanford.edu/~cgates/PERSI/papers/bayer92.pdf)